# Обработчик Метеорологических Зондирований - Онлайн Демонстрация

Этот ноутбук демонстрирует использование инструмента для обработки метеорологических зондирований без необходимости устанавливать что-либо на ваш компьютер.

[![Открыть в Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meteoFurletov/weather-sounding-tool/blob/main/demo_ru.ipynb)

## 1. Настройка

Сначала установим необходимые библиотеки и клонируем репозиторий:

In [ ]:
# Установка необходимых библиотек
!pip install pandas numpy requests beautifulsoup4 xlsxwriter matplotlib

# Клонирование репозитория
!git clone https://github.com/meteoFurletov/weather-sounding-tool.git
# Переход в директорию репозитория
%cd weather-sounding-tool

## 2. Выбор параметров

Настроим основные параметры для загрузки и обработки данных:

In [ ]:
# Параметры для загрузки данных
year = 2024  # Год
month = 1    # Месяц (1-12)

# Список популярных станций для справки
stations = {
    26075: "Санкт-Петербург (Воейково)",
    26702: "Москва",
    26850: "Казань",
    27612: "Екатеринбург",
    31977: "Салехард",
    32540: "Новосибирск",
    35121: "Владивосток"
}

# Ввод ID станции
station_id = 26075

print(f"Выбраны параметры: {year} год, {month} месяц, станция: {station_id}")

## 3. Загрузка данных метеорологического зондирования

Теперь загрузим данные зондирования с сайта Университета Вайоминга:

In [ ]:
# Импортируем необходимые функции
from weather_tool import fetch_data

# Загрузка данных
print(f"Загрузка данных для станции {station_id} за {month}.{year}...")
file_path = fetch_data(year, month, station_id)

if file_path:
    print(f"Данные успешно загружены и сохранены в {file_path}")
else:
    print("Ошибка при загрузке данных. Проверьте соединение или параметры.")

## 4. Обработка данных

Теперь обработаем загруженные данные, чтобы извлечь температурные инверсии:

In [ ]:
from weather_tool import process_data

# Обработка данных
print(f"Обработка данных для станции {station_id}...")
result = process_data(year, [month], station_id, use_local_files=True)

if result['processed'] > 0:
    print(f"Обработано {result['processed']} зондирований")
    print(f"Найдено инверсий: {len(result['files'])}")
    print(f"Не найдено инверсий в {result['failed']} зондированиях")
else:
    print("Не удалось найти инверсии. Проверьте данные или параметры.")

## 5. Просмотр результатов

Рассмотрим результаты анализа:

In [ ]:
import pandas as pd

# Загружаем данные из сводного файла
df = pd.read_excel('DATA.xlsx', sheet_name="df_full")

# Показываем первые несколько строк
print("Первые строки данных:")
df.head()

## 6. Основные статистические показатели

Рассчитаем базовую статистику по инверсиям:

In [ ]:
# Отфильтруем строки, где есть данные (не NaN)
df_clean = df.dropna(subset=['ΔT'])

if len(df_clean) > 0:
    print(f"Всего инверсий: {len(df_clean)}")
    print(f"Средняя интенсивность инверсии (ΔT): {df_clean['ΔT'].mean():.2f}°C")
    print(f"Максимальная интенсивность: {df_clean['ΔT'].max():.2f}°C")
    print(f"Средняя мощность инверсии (ΔH): {df_clean['ΔH'].mean():.2f} м")
    
    ground_inv = df_clean[df_clean['Ground'] == 1]
    elevated_inv = df_clean[df_clean['Ground'] == 0]
    
    print(f"\nНаземных инверсий: {len(ground_inv)} ({len(ground_inv)/len(df_clean)*100:.1f}%)")
    print(f"Приподнятых инверсий: {len(elevated_inv)} ({len(elevated_inv)/len(df_clean)*100:.1f}%)")
    
    print(f"\nСредняя интенсивность наземных инверсий: {ground_inv['ΔT'].mean():.2f}°C")
    print(f"Средняя интенсивность приподнятых инверсий: {elevated_inv['ΔT'].mean():.2f}°C")
else:
    print("Нет данных для анализа")

## 7. Визуализация результатов

Визуализируем обнаруженные инверсии:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

if len(df_clean) > 0:
    plt.figure(figsize=(12, 8))
    
    # График интенсивности инверсий по времени
    plt.subplot(211)
    ground = df_clean[df_clean['Ground'] == 1]
    elevated = df_clean[df_clean['Ground'] == 0]
    
    plt.scatter(ground['date'], ground['ΔT'], color='red', label='Наземные инверсии', alpha=0.7)
    plt.scatter(elevated['date'], elevated['ΔT'], color='blue', label='Приподнятые инверсии', alpha=0.7)
    
    plt.title(f'Интенсивность инверсий, {station_id}, {month}.{year}')
    plt.ylabel('ΔT (°C)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))
    
    # График мощности инверсий
    plt.subplot(212)
    plt.scatter(ground['date'], ground['ΔH'], color='red', label='Наземные инверсии', alpha=0.7)
    plt.scatter(elevated['date'], elevated['ΔH'], color='blue', label='Приподнятые инверсии', alpha=0.7)
    
    plt.title('Мощность инверсий')
    plt.xlabel('Дата')
    plt.ylabel('ΔH (м)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))
    
    plt.tight_layout()
    plt.show()
else:
    print("Недостаточно данных для построения графика")

## 8. Сравнение дневных и ночных инверсий

In [ ]:
if len(df_clean) > 0:
    day = df_clean[df_clean['Day'] == 1]
    night = df_clean[df_clean['Night'] == 1]
    
    print(f"Дневных инверсий: {len(day)}")
    print(f"Ночных инверсий: {len(night)}")
    
    if len(day) > 0 and len(night) > 0:
        print("\nСравнение интенсивности:")
        print(f"Средняя ΔT днём: {day['ΔT'].mean():.2f}°C")
        print(f"Средняя ΔT ночью: {night['ΔT'].mean():.2f}°C")
        
        print("\nСравнение мощности:")
        print(f"Средняя ΔH днём: {day['ΔH'].mean():.2f} м")
        print(f"Средняя ΔH ночью: {night['ΔH'].mean():.2f} м")
        
        # Визуализация сравнения
        plt.figure(figsize=(10, 6))
        
        # Интенсивность
        labels = ['Дневные', 'Ночные']
        dt_values = [day['ΔT'].mean(), night['ΔT'].mean()]
        dh_values = [day['ΔH'].mean(), night['ΔH'].mean()]
        
        x = range(len(labels))
        width = 0.35
        
        fig, ax1 = plt.subplots(figsize=(8, 6))
        
        ax1.bar(x, dt_values, width, label='ΔT (°C)', color='tomato')
        ax1.set_ylabel('ΔT (°C)', color='tomato')
        ax1.tick_params(axis='y', labelcolor='tomato')
        
        ax2 = ax1.twinx()
        ax2.bar([i + width for i in x], dh_values, width, label='ΔH (м)', color='royalblue')
        ax2.set_ylabel('ΔH (м)', color='royalblue')
        ax2.tick_params(axis='y', labelcolor='royalblue')
        
        plt.title('Сравнение дневных и ночных инверсий')
        plt.xticks([i + width/2 for i in x], labels)
        plt.tight_layout()
        plt.show()
else:
    print("Недостаточно данных для анализа")

## 9. Загрузка результатов

Вы можете скачать сводную таблицу с результатами:

In [ ]:
from google.colab import files

# Загрузка сводного файла анализа
files.download('DATA.xlsx')

## Заключение

В этом ноутбуке мы:
1. Загрузили данные метеорологических зондирований
2. Обработали их для выявления температурных инверсий
3. Проанализировали характеристики инверсий
4. Визуализировали результаты
5. Сравнили дневные и ночные инверсии

Для дальнейшего анализа вы можете:
- Изменить период анализа (год, месяц)
- Выбрать другую метеостанцию, используя любой ID станции
- Сравнить данные между несколькими станциями
- Использовать более сложный статистический анализ

Примечание: Полный список станций и их ID можно найти на сайте Университета Вайоминга: http://weather.uwyo.edu/upperair/sounding.html